In [1]:
import numpy as np

# UR Interface
import rtde_control
import rtde_receive

# Spatial Math is used for manipulating geometric primitives
import spatialmath as sm
from spatialmath import SE3

# Poses is from rmlib and used for converting between 4 x 4 homogenous pose and 6 element vector representation (x,y,z,rx,ry,rz)
import poses


In [2]:
def pose_vector_to_homog_coord( poseVec ):
    """ Express the pose vector in homogeneous coordinates """
    # poseVector is a 6 element list of [x, y, z, rX, rY, rZ]
    return sm.SE3( poses.pose_vec_to_mtrx( poseVec ) )


def homog_coord_to_pose_vector( poseMatrix ):
    """ Converts poseMatrix into a 6 element list of [x, y, z, rX, rY, rZ] """
    # poseMatrix is a SE3 Object (4 x 4 Homegenous Transform) or numpy array
    return poses.pose_mtrx_to_vec( np.array( poseMatrix ) )
    
    

In [3]:
pose_vector_to_homog_coord( [-0.277, -0.304, 0.271, 2.256, 2.218, -0.028] )

   0.01703   0.9995   -0.02821  -0.277     
   0.9999   -0.01694   0.003451 -0.304     
   0.002971 -0.02826  -0.9996    0.271     
   0         0         0         1         


In [4]:
class UR5_Interface:
    """ Interface class to `ur_rtde` """
    
    def __init__( self, robotIP = "192.168.0.6" ):
        """ Store connection params """
        self.robotIP  = robotIP # IP address of the robot
        self.ctrl     = None # -- `RTDEControlInterface` object 
        self.recv     = None # -- `RTDEReceiveInterface` object 
        self.gripper  = None # -- Gripper Controller Interface
        self.homePose = None
        # self.homePose = np.array( # 2023-05-04: This pose is scary
        #        [[ 0.0, 1.0,  0.0, -0.277 ],   
        #         [ 1.0, 0.0,  0.0, -0.304 ],    
        #         [ 0.0, 0.0, -1.0,  0.271 ],     
        #         [ 0.0, 0.0,  0.0,  1.0   ]]
        # )
        # self.homePose = np.array( # 2023-05-04: This pose is scary
        #     [[ 0.99955322, -0.02418213, -0.01756664,  0.01498893],
        #      [-0.01748495,  0.00358545, -0.9998407 , -0.57686779],
        #      [ 0.02424126,  0.99970114,  0.00316103,  0.05545535],
        #      [ 0.0       ,  0.0       ,  0.0       ,  1.0       ]]
        # )
        
        
    def start( self ):
        """ Connect to RTDE and the gripper """
        self.ctrl = rtde_control.RTDEControlInterface( self.robotIP )
        self.recv = rtde_receive.RTDEReceiveInterface( self.robotIP )
        
        
    def stop( self ):
        """ Shutdown robot and gripper connections """
        self.ctrl.servoStop()
        self.ctrl.stopScript()
        
        
    def get_joint_angles( self ):
        """ Returns a 6 element numpy array of joint angles (radians) """
        thetas = np.array( self.recv.getActualQ() )
        return thetas
    
    
    def get_tcp_pose( self ):
        """ Returns the current pose of the gripper as a SE3 Object (4 x 4 Homegenous Transform) """
        return sm.SE3( pose_vector_to_homog_coord( self.recv.getActualTCPPose() ) )
    
    
    def moveJ( self, qGoal, rotSpeed = 1.05, rotAccel = 1.4, blocking = True ):
        """ qGoal is a 6 element numpy array of joint angles (radians) """
        # speed is joint velocity (rad/s)
        self.ctrl.moveJ( list( qGoal ), rotSpeed, rotAccel, blocking )
    
    
    def moveL( self, poseMatrix, linSpeed = 0.25, linAccel = 0.5, blocking = True ):
        """ Moves tool tip pose linearly in cartesian space to goal pose (requires tool pose to be configured) """
        # poseMatrix is a SE3 Object (4 x 4 Homegenous Transform) or numpy array
        # tool pose defined relative to the end of the gripper when closed
        self.ctrl.moveL( homog_coord_to_pose_vector( poseMatrix ), linSpeed, linAccel, blocking )
    
    
    def move_home( self, linSpeed = 0.25, linAccel = 0.5, blocking = True ):
        # Moves the arm linearly in cartesian space to home pose
        self.moveL( self.homePose, linSpeed, linAccel, blocking )
        # self.arm.move(target=homePose,move_type="l")
    
    
    def openGripper( self ):
        self.gripper.openGripper()
    
    
    def closeGripper( self, width = 10 ):
        # Computes the servo angles needed for the jaws to be width mm apart
        # Sends command over serial to the gripper to hold those angles
        dTheta = self.gripper.distance2theta(width)
        self.gripper.position(dTheta)
        
        
    def testRoutine( self ):
        # Moves ur +1 cm in world frame z-axis and then opens, closes, and opens gripper 
        print("Running Test Routine")
        initPose = np.array(self.getPose())
        print(f"TCP Pose:\n{initPose}")
        dX,dY,dZ = 0,0,2/100 # in m
        goalPose = initPose
        goalPose[2][3] += dZ
        goalPose = sm.SE3(goalPose)
        print(f"Goal TCP Pose:\n{goalPose}")
        self.moveL(sm.SE3(goalPose))
        print(f"Final TCP Pose:\n{self.getPose()}")
        print("Opening Gripper")
        self.openGripper()
        time.sleep(1)
        self.ctrlloseGripper(10)
        time.sleep(2)
        self.openGripper()

In [5]:
robot = UR5_Interface()

In [6]:
robot.start()

In [7]:
robot.move_home()

In [ ]:
robot.stop()